# IMDb 文本情感分析

## RNN分析——深度学习初体验（坐牢）

In [1]:
#测试cuda是否可用
import torch
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available.")

CUDA is available.


In [2]:
import warnings
warnings.filterwarnings("ignore")
#导入数据
import pandas as pd
data = pd.read_csv('E:/本科/数据挖掘与商务分析/hw/final/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
#将数据集分为训练集和测试集
from sklearn.model_selection import train_test_split
X = data['review']
y = data['sentiment']
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
test_dataset = [(x, y) for x, y in zip(X_test, y_test)]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state=1)
train_dataset = [(x, y) for x, y in zip(X_train, y_train)]
valid_dataset = [(x, y) for x, y in zip(X_valid, y_valid)]

In [4]:
import re
from collections import Counter, OrderedDict

token_counts = Counter()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized


for review, _ in train_dataset:
    tokens = tokenizer(review)
    token_counts.update(tokens)
 
    
print('Vocab-size:', len(token_counts))

Vocab-size: 71093


In [5]:
#利用torchtext的vocab将token转换为整数
from torchtext.vocab import vocab

sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True) 
ordered_dict = OrderedDict(sorted_by_freq_tuples)

vocab = vocab(ordered_dict)
vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.set_default_index(1)

print([vocab[token] for token in ['this', 'is', 'an', 'example']])

[11, 7, 35, 479]


In [6]:
#定义转换函数
import torch.nn as nn
device = torch.device('cuda' )
review_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
sentiment_pipeline = lambda x: 1 if x == 'positive' else 0

def collate_batch(batch):
    review_list, sentiment_list, lengths = [], [], []
    for _review, _sentiment,in batch:
        processed_text =torch.tensor(review_pipeline(_review), dtype=torch.int64)
        review_list.append(processed_text)
        sentiment_list.append(sentiment_pipeline(_sentiment))
        lengths.append(processed_text.size(0))
    sentiment_list = torch.tensor(sentiment_list)
    lengths = torch.tensor(lengths)
    padded_review_list = nn.utils.rnn.pad_sequence(review_list, batch_first=True)
    return padded_review_list.to(device), sentiment_list.to(device), lengths.to(device)

In [7]:
#small batch 看看实力
from torch.utils.data import DataLoader
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False, collate_fn=collate_batch)
review_batch, sentiment_batch, length_batch = next(iter(dataloader))
print(review_batch)
print(sentiment_batch)
print(length_batch)
print(review_batch.shape)

tensor([[   16, 20203,  4622,  ...,     0,     0,     0],
        [43943,     7,     3,  ...,     0,     0,     0],
        [   10,   142,     3,  ...,  1092,     6,    31],
        [  310,  1691,  2179,  ...,     0,     0,     0]], device='cuda:0')
tensor([0, 1, 1, 1], device='cuda:0')
tensor([270, 364, 416, 113], device='cuda:0')
torch.Size([4, 416])


In [8]:
#batch data生成data loader
batch_size = 32 

train_dl = DataLoader(train_dataset, batch_size=batch_size,shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size,shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [9]:
#将句子转为嵌入编码
embedding = nn.Embedding(num_embeddings=10, embedding_dim=3, padding_idx=0)

#example
text_encoded_input = torch.LongTensor([[1,2,4,5],[4,3,2,0]])
print(embedding(text_encoded_input))

tensor([[[ 2.7666,  1.4503, -1.3632],
         [-1.6674,  1.0552,  0.3405],
         [ 0.0287, -0.1377,  0.5142],
         [-0.2761, -0.0186, -2.4675]],

        [[ 0.0287, -0.1377,  0.5142],
         [-0.5876, -1.4813, -0.6294],
         [-1.6674,  1.0552,  0.3405],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward0>)


### 构建循环神经网络

In [10]:
#用RNN进行情感分析
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim, 
                                      padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)
model

RNN(
  (embedding): Embedding(71095, 20, padding_idx=0)
  (rnn): LSTM(20, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [11]:
#编写train函数
def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        text_batch = text_batch.to(device)
        label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
        lengths = lengths.to(device)
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)
 
def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
            lengths = lengths.to(device)
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
            # 收集所有预测分数和真实标签
            all_labels.extend(label_batch.cpu().numpy())
            all_preds.extend(pred.cpu().numpy())
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset), all_labels, all_preds

In [12]:
#编写损失函数
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10 

torch.manual_seed(1)

#开始显卡炼丹 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid,_,_ = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')


Epoch 0 accuracy: 0.5987 val_accuracy: 0.5022
Epoch 1 accuracy: 0.7109 val_accuracy: 0.7166
Epoch 2 accuracy: 0.7269 val_accuracy: 0.7724
Epoch 3 accuracy: 0.8294 val_accuracy: 0.8200
Epoch 4 accuracy: 0.8887 val_accuracy: 0.8234
Epoch 5 accuracy: 0.9138 val_accuracy: 0.8274
Epoch 6 accuracy: 0.9375 val_accuracy: 0.8600
Epoch 7 accuracy: 0.9526 val_accuracy: 0.8650
Epoch 8 accuracy: 0.9678 val_accuracy: 0.8706
Epoch 9 accuracy: 0.9778 val_accuracy: 0.8656
Epoch 10 accuracy: 0.9846 val_accuracy: 0.8672
Epoch 11 accuracy: 0.9905 val_accuracy: 0.8628
Epoch 12 accuracy: 0.9919 val_accuracy: 0.8540
Epoch 13 accuracy: 0.9946 val_accuracy: 0.8622
Epoch 14 accuracy: 0.9951 val_accuracy: 0.8608
Epoch 15 accuracy: 0.9959 val_accuracy: 0.8596
Epoch 16 accuracy: 0.9967 val_accuracy: 0.8642
Epoch 17 accuracy: 0.9980 val_accuracy: 0.8648
Epoch 18 accuracy: 0.9962 val_accuracy: 0.8658
Epoch 19 accuracy: 0.9982 val_accuracy: 0.8618
Epoch 20 accuracy: 0.9987 val_accuracy: 0.8596
Epoch 21 accuracy: 0.99

In [13]:
#用sklearn评估测试集效果
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
acc_test, loss_test, true_labels, pred_scores= evaluate(test_dl)

# 二分类的阈值设置为0.5
pred_labels = [1 if score >= 0.5 else 0 for score in pred_scores]
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)
auc = roc_auc_score(true_labels, pred_scores)
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'AUC Score: {auc:.3f}')

Precision: 0.851
Recall: 0.898
F1 Score: 0.874
AUC Score: 0.932


### 双向循环神经网络

In [29]:
#用RNN进行情感分析
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim, 
                                      padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(rnn_hidden_size*2, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn(out)
        out = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)
model

RNN(
  (embedding): Embedding(71095, 20, padding_idx=0)
  (rnn): LSTM(20, 64, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [30]:
#编写train函数
def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        text_batch = text_batch.to(device)
        label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
        lengths = lengths.to(device)
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)
 
def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
            lengths = lengths.to(device)
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
            # 收集所有预测分数和真实标签
            all_labels.extend(label_batch.cpu().numpy())
            all_preds.extend(pred.cpu().numpy())
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset), all_labels, all_preds

In [28]:
#编写损失函数
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10 

torch.manual_seed(1)

#开始显卡炼丹 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid,_,_ = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')

Epoch 0 accuracy: 0.6117 val_accuracy: 0.6420
Epoch 1 accuracy: 0.6414 val_accuracy: 0.7438
Epoch 2 accuracy: 0.8025 val_accuracy: 0.8212
Epoch 3 accuracy: 0.8752 val_accuracy: 0.8482
Epoch 4 accuracy: 0.9109 val_accuracy: 0.8534
Epoch 5 accuracy: 0.9301 val_accuracy: 0.8362
Epoch 6 accuracy: 0.9507 val_accuracy: 0.8698
Epoch 7 accuracy: 0.9637 val_accuracy: 0.8652
Epoch 8 accuracy: 0.9772 val_accuracy: 0.8670
Epoch 9 accuracy: 0.9850 val_accuracy: 0.8718


In [17]:
#用sklearn评估测试集效果
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
acc_test, loss_test, true_labels, pred_scores= evaluate(test_dl)

# 二分类的阈值设置为0.5
pred_labels = [1 if score >= 0.5 else 0 for score in pred_scores]
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)
auc = roc_auc_score(true_labels, pred_scores)
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'AUC Score: {auc:.3f}')

Precision: 0.891
Recall: 0.805
F1 Score: 0.846
AUC Score: 0.932
